In [12]:
import pandas as pd
import numpy as np

In [13]:
def draw_sudoku(board):
    print("+" + "---+"*9)
    for i, row in enumerate(board):
        print(("|" + " {}   {}   {} |"*3).format(*[x if x != 0 else "0" for x in row]))
        if i % 3 == 2:
            print("+" + "---+"*9)

In [14]:
def read_csv(name):
    df = pd.read_csv(name, header=None)
    matrix = df.to_numpy()
    d=[]
    x = 0
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if(matrix[i,j] > 0):
                d.append( [[i,j],matrix[i,j]])
                x+=1
    print(fitness(matrix))
    draw_sudoku(matrix)
    return matrix, d

In [15]:
def initial_population(arr, n):    # generate n random matrix with values from 1->9
    x=y=val=-1
    population = []
    for i in range(n):
        population.append(np.zeros((9,9)))
    for j in range(n):
        mat = np.random.randint(low=1,high=10, size=(9, 9))
        #print(mat)
        for i in range(len(arr)):
            x = arr[i][0][0]
            y = arr[i][0][1]
            val = arr[i][1]
            mat[x,y] = val
        population[j] = mat
    return population

In [16]:
def fitness(m):
    side = m.shape[0]
    def repeated(st):
        st, c = np.unique(st, return_counts = True)
        if st[0] != 0: return np.count_nonzero(c > 1)
        return np.count_nonzero(c[1:] > 1)
    result = side**2
    for i in range(side):
        result -= repeated(m[i, :])
        result -= repeated(m[:, i])
        sqrt = np.sqrt(side)
        h, v = (i % sqrt)*sqrt, (i // sqrt)*sqrt
        result -= repeated(m[int(v): int(v + sqrt), int(h): int(h + sqrt)])
    return result - np.count_nonzero(m == 0)

In [17]:
def tournament(fitnesses, k = 2, reversed = False):
    participants = np.random.choice(len(fitnesses), k, False)
    if reversed: return np.argmin(fitnesses[participants])
    return np.argmax(fitnesses[participants])

In [18]:
def simplePermutationCrossover(pR, parents, d):
    if np.random.uniform(0, 1) < pR:
        return crossover(parents, d)
    return parents[0]

In [19]:
def crossover(parents, d):
    d = d.copy()
    first, second = parents
    x = np.random.randint(low=0,high=9)
    offspring = np.zeros((9,9))
    offspring = offspring.astype(int)

    x=4 
    # merge matrixes
    for i in range(x+1):
        for j in range(offspring.shape[0]):
                offspring[i][j] = first[i][j]
    for i in range(x+1, offspring.shape[0]):
        for j in range(offspring.shape[1]):
                offspring[i][j] = second[i][j]
    #print(offspring)

    # missing-duplicated
    blocked = []
    for i in range(offspring.shape[0]):
        coords = []
        dupl = {}
        miss = [1,2,3,4,5,6,7,8,9]
        
        for j in range(offspring.shape[1]):
            coords.append([offspring[j,i], [i,j]])
        # print(coords)
        for j in range(len(coords)):
            rep = np.where(np.array(coords == coords[j][0], dtype=object))
            if(rep[0].shape[0] != 1):
                dupl[coords[j][0]] = rep[0]
        for j in range(len(coords)):
            if(coords[j][0] in miss):
                miss.remove(coords[j][0])
        for j in d:
            blocked.append(j[0])
        for j in dupl.values():
            for z in j:
                if([z,i] not in blocked): # si no está en la lista de bloqueados
                    # print([z,i], miss )
                    if(len(miss)):
                        offspring[z,i] = miss[0]
                        miss.pop(0)
    return offspring 

In [20]:
def mutation(pM, m):
    if np.random.uniform(0, 1) < pM:
        sqrt, side = int(np.sqrt(m.shape[0])), m.shape[0]
        for i in range(side):
            _, indices = np.unique(m[i], return_index=True)
            for j in range(side):
                if j not in indices and m[i, j]:
                    m[i, j], m[(i + sqrt) % side, j] = m[(i + sqrt) % side, j], m[i, j]
    return m

In [21]:
r = np.zeros((9,9))
def geneticAlgorithm(f, n, g, pR, pM): #File name, Population Size, # of generations, P(reproduction), p(Mutation), # of queens
    m, d = read_csv(f)
    sideSize = m.shape[0]
    population = initial_population(d, n)
    best, fitnesses = -1, np.array([fitness(i) for i in population])
    best = np.argmax(fitnesses)
    for i in range(g):
        if i % 50 == 0: 
            r = population[best].astype(int)
            #print(fitness(r))
            #draw_sudoku(r)
        if fitnesses[best] == sideSize**2:
            return population[best].astype(int)

        newPopulation, newFitnesses = np.zeros((n, sideSize, sideSize)), np.zeros(n)
        for childIndex in range(n):
            pOne = population[tournament(fitnesses.copy(), n // 2)].copy()
            pTwo = population[tournament(fitnesses.copy(), n // 2)].copy()
            #pOne = population[roulette(population)]
            #pTwo = population[roulette(population)]
            parents = np.array([pOne, pTwo])
            newChild = simplePermutationCrossover(pR, parents, d)
            newChild = mutation(pM, newChild)
            newFitnesses[childIndex] = fitness(newChild)
            newPopulation[childIndex] = newChild
        betterBest = np.argmax(newFitnesses)
        #print(fitnesses[best] ,newFitnesses[betterBest])
        #print(fitnesses, newFitnesses)
        #print("Before:", best, "->", fitnesses[best], betterBest, "->", newFitnesses[betterBest])
        #print(fitnesses, newFitnesses)
        if newFitnesses[betterBest] < fitnesses[best]: 
            #print("Seeded")
            newPopulation[tournament(fitnesses.copy(), n//2, True)] = population[best].copy()
            newFitnesses[best] = fitnesses[best].copy()
        else:
            #print("Updated")
            best = betterBest
        population, fitnesses = newPopulation.copy(), newFitnesses.copy()
        #print("After:", best, "->", fitnesses[best], betterBest, "->", newFitnesses[betterBest])
        #print(fitnesses, newFitnesses)
        print("Current Best", fitnesses[best])
    return population[best].astype(int)
geneticAlgorithm('easy2.csv', 1000, 100000, 0.75, 0.75)
print(fitness(r))
draw_sudoku(r)

38
+---+---+---+---+---+---+---+---+---+
| 0   9   8 | 2   0   0 | 4   0   6 |
| 0   1   5 | 3   9   4 | 8   0   0 |
| 4   2   0 | 0   0   0 | 5   1   0 |
+---+---+---+---+---+---+---+---+---+
| 0   0   0 | 4   7   0 | 1   0   0 |
| 1   0   7 | 0   8   0 | 0   9   3 |
| 0   0   6 | 0   3   0 | 0   0   5 |
+---+---+---+---+---+---+---+---+---+
| 8   0   0 | 0   0   3 | 6   7   1 |
| 3   0   4 | 6   0   8 | 0   0   0 |
| 9   0   0 | 0   0   5 | 0   8   0 |
+---+---+---+---+---+---+---+---+---+


/tmp/ipykernel_2711/1061474561.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  rep = np.where(np.array(coords == coords[j][0], dtype=object))


Current Best 55.0
Current Best 55.0
Current Best 55.0
Current Best 55.0
Current Best 55.0
Current Best 55.0


KeyboardInterrupt: 